# Sentimental Analysis

The following library are to be explored:

1. VADER
2. Hugging Face Transformers (Roberta - pretrained model)
3. TextBlob
4. Ntlk
5. spaCY
6. HuggingFace Transformers
7. API LLM/GPT
8. Flair
9. DistilBERT

In [ ]:
# setup
test_sentence = "Trump to Leave G-7 Tonight Due to Middle East Crisis"

### VADER

In [ ]:
# pip install vadersentiment

In [ ]:
# Prebuilt Vader sentiment package

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
scores = analyzer.polarity_scores(test_sentence)
print(scores)

### TextBlob

In [ ]:
# pip install textblob

In [ ]:
# Prebuilt Textblob sentiment package

from textblob import TextBlob
text = TextBlob(test_sentence)
score = text.sentiment
print(score)

### HuggingFace Transformers (RoBERTa - a pretrained model)

RoBERTa (using a Pretrained Model via HuggingFace Transformers)

In [ ]:
# pip install transformers

In [1]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
text = "This movie is great!"
result = classifier(text)
print(result)

C:\Users\Jay Tai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\Jay Tai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\c10.dll" or one of its dependencies.

In [4]:
pip uninstall torch torchvision torchaudio

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu